<a href="https://colab.research.google.com/github/hoonheelee-coder/danuri/blob/main/Making_My_Own_AI_Lunar_Orbiter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 거대언어모델(LLM)을 이용한 나만의 AI 달 탐사선

In [ ]:
import torch; torch.__version__

'2.3.0+cu121'

In [ ]:
!pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-602p1wwg/unsloth_195c059164cf45aaa00a9f3f39c86666
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-602p1wwg/unsloth_195c059164cf45aaa00a9f3f39c86666
  Resolved https://github.com/unslothai/unsloth.git to commit 92dce38e8b3c1db209cef860d90b60188e95f0f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s e

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="beomi/Llama-3-Open-Ko-8B-Instruct-preview",  # Llama-3 8B 기반의 모델 이용
    max_seq_length=4096,  # 모델에 입력되는 질문의 최대 길이
    dtype=None,  # 데이터 타입
    load_in_4bit=True,  # 메모리 절약
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B-Instruct-preview does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=123,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = load_dataset("hoon-hee-lee/danuri", split="train")

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Generating train split:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        max_steps=100,
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=123,
        output_dir="outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
1,3.470400,3.437026
2,3.506700,3.238483
3,3.202300,2.881438
4,2.984400,2.479722
5,2.453200,2.001850
6,2.128000,1.573490
7,1.554200,1.350185
8,1.432000,1.158695
9,1.145900,0.988023
10,1.022000,0.800583


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )

stop_token = "<|end_of_text|>"
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]


stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "다누리에 대해서 알려주세요.",  # Instruction
            "",  # Output
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,
    stopping_criteria=stopping_criteria
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
다누리에 대해서 알려주세요.
### Response:
다누리는 대한민국 최초의 경차인 티코보다도 무거운 약 687kg의 무게를 가지고 있고 전기모터로만 구동되며 최고속도는 약 121km/h입니다.<|end_of_text|>


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "대한민국의 다누리는 달에 가기 위해 어떤 발사체를 이용했니?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다.
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
대한민국의 다누리는 달에 가기 위해 어떤 발사체를 이용했니?
### Response:
대한민국의 다누리는 2022년 8월 5일 미국의 스페이스 X사의 팰컨 9 발사체에 실려 발사되었습니다.<|end_of_text|>


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "이훈희 교수님이 왜 지금 제주도에 있나요?",  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
이훈희 교수님이 왜 지금 제주도에 있나요?
### Response:
이훈희 교수님은 한국항공우주연구원에서 인공지능 연구를 진행하였고 제주도의 대학생들이 인공지능 공부에 소외되지 않도록 도와주기 위해서 왔습니다.<|end_of_text|>


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "NASA의 섀도우캠이 다누리에 들어가 있는 이유가 뭔가요?",  # 지시사항
            "",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
NASA의 섀도우캠이 다누리에 들어가 있는 이유가 뭔가요?
### Response:
섀도우캠은 약 1.7m급의 달 영구음영지역 사진을 촬영하기 위한 목적으로 다누리호에 실려 발사되었습니다.  영구음영지역은 주로 달 남북극 지역에 위치하고 있고 물과 같은 중요한 물질의 부존량 조사가 필요합니다.<|end_of_text|>


In [ ]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "다누리가 사용한 달 궤도 전이 방식은 무엇인가요?",  # 지시사항
            "",  # 출력 - 앞부분의 힌트 제공 예시
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
다누리가 사용한 달 궤도 전이 방식은 무엇인가요?
### Response:
다누리는 달 궤도에 접근하는 달 궤도 전이 방식을 선택하여 달로 이동했으며 그 방식은 달 궤도에 있는 달 중간 궤도(약 125,000 km)까지 비행하여 궤도에너지를 증가하여 달에 포획될 때의 속도 증분(ΔV)을 약 25% 정도 감소시킬 수 있는 궤적으로 탐사선의 연료 소모량을 최소로 사용하기 위해서 선택한 방법입니다.<|end_of_text|>
